# Creating a Registered Model Version from an experiment run on Verta

Within Verta, a "Model" can be any arbitrary function: a traditional ML model (e.g., sklearn, PyTorch, TF, etc); a function (e.g., squaring a number, making a DB function etc.); or a mixture of the above (e.g., pre-processing code, a DB call, and then a model application.) See more [here](https://docs.verta.ai/verta/registry/concepts).

This notebook provides an example of how to create a Registered Model Version on Verta from an Experiment Run. Detailed docs are available here (https://verta.readthedocs.io/en/master/_autogen/verta.registry.entities.RegisteredModel.html).

## 0. Imports

In [1]:
from __future__ import print_function

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import os
import time

import six

import numpy as np
import pandas as pd

import sklearn
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

### 0.1 Verta import and setup

In [2]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [3]:
import os

# Ensure credentials are set up, if not, use below
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST'] = 

from verta import Client

client = Client(os.environ['VERTA_HOST'])

## 1. Model Training

### 1.1 Load training data

In [4]:
try:
    import wget
except ImportError:
    !pip install wget  # you may need pip3
    import wget

In [5]:
train_data_url = "http://s3.amazonaws.com/verta-starter/census-train.csv"
train_data_filename = wget.detect_filename(train_data_url)
if not os.path.isfile(train_data_filename):
    wget.download(train_data_url)

test_data_url = "http://s3.amazonaws.com/verta-starter/census-test.csv"
test_data_filename = wget.detect_filename(test_data_url)
if not os.path.isfile(test_data_filename):
    wget.download(test_data_url)

In [6]:
df_train = pd.read_csv(train_data_filename)
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv(test_data_filename)
X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:, -1]


df_train.head()

### 1.2 Train model

In [7]:
proj = client.set_project("Census Registry Example")
expt = client.set_experiment("My Experiment")
run = client.set_experiment_run()
hyperparams = {'C' : 10, 'solver' : 'lbfgs'}
run.log_hyperparameters(hyperparams)

# create validation split
(X_val_train, X_val_test,
 y_val_train, y_val_test) = model_selection.train_test_split(X_train, y_train,
                                                             test_size=0.2,
                                                             shuffle=True)

# create and train model
model = linear_model.LogisticRegression(**hyperparams)
model.fit(X_train, y_train)

# calculate and log validation accuracy
val_acc = model.score(X_val_test, y_val_test)
run.log_metric("val_acc", val_acc)

## 2. Register a model

In [8]:
registered_model = client.get_or_create_registered_model(
    name="census-registry-example", labels=["research-purpose", "team-a"])

In [9]:
from verta.environment import Python

model_version = registered_model.create_version_from_run(
    run.id
)

### Note: because an ER may not have the required fields, this registered model version is not tagged as a Verta Standard Model

---